In [2]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
from astropy import units as u
from regions import Regions

def create_cutout_4d(fits_file, ra, dec, freq_idx, stokes_idx, size, output_file):
    # Open the FITS file
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        data = hdul[0].data[0,0 , :, :]
        
        wcs = WCS(header)

        # Create a Cutout2D object
        cutout = Cutout2D(data, (ra,dec), size)
        
        # Update header information for the cutout
        #cutout_header = cutout.wcs.to_header()
        cutout_data = cutout.data

        # turn pixel coordinates into world coordinates
        wcs = WCS(header)
        
        # deg from pixel coordinates
        vals = wcs.wcs_pix2world([[ra,dec,0,0]],0)
      
        ra = vals[0][0]
        dec = vals[0][1]

        # Save the cutout to a new FITS file
        cutout_header = header.copy()
        cutout_header['NAXIS'] = 2
        cutout_header['NAXIS1'] = size[0]
        cutout_header['NAXIS2'] = size[1]
        cutout_header['CRPIX1'] = size[0]/2
        cutout_header['CRPIX2'] = size[1]/2
        cutout_header['CRVAL1'] = ra # ra of source center # allows the ra and dec to be consistent. (for truth cat)
        cutout_header['CRVAL2'] = dec # dec of source center 

        # remove the 3rd and 4th axis
        cutout_header.remove('NAXIS3')
        cutout_header.remove('NAXIS4')
        cutout_header.remove('CRPIX3')
        cutout_header.remove('CRPIX4')
        cutout_header.remove('CDELT3')
        cutout_header.remove('CDELT4')
        cutout_header.remove('CTYPE3')
        cutout_header.remove('CTYPE4')
        cutout_header.remove('CRVAL3')
        cutout_header.remove('CRVAL4')

        cutout_hdu = fits.PrimaryHDU(data=cutout_data, header=cutout_header)

        hdulist = fits.HDUList([cutout_hdu])
        hdulist.writeto(output_file, overwrite=True)
    
    

# Specify the paths to your FITS file and output file name
fits_file = '/Users/rs17612/Documents/Radio_Data/SKA_Challenge_1/SKAMid_B1_1000h_v3.fits'
output_file = '/Users/rs17612/Documents/Radio_Data/SKA_Challenge_1/output_cutout.fits'
region_file = '/Users/rs17612/Documents/Radio_Data/SKA_Challenge_1/AGN-interested_soruces'
crtfslist = Regions.read(region_file,format='crtf')

for crtfs in crtfslist:
   
    # Specify the central RA and Dec of the cutout in degrees
    ra = crtfs.center.x
    dec = crtfs.center.y

    # Specify the indices for the frequency and Stokes axes
    freq_idx = 0
    stokes_idx = 0

    # Specify the size of the cutout in pixels

    width = crtfs.width
    height = crtfs.height

    # convert width and heigh in pixels using wcs
    cutout_size = (height, width)

    # Call the function to create the cutout and save it
    output_filename = '/Users/rs17612/Documents/Radio_Data/SKA_Challenge_1/cutout_'+str(crtfs.center.x)+'.fits'
    create_cutout_4d(fits_file, ra, dec, freq_idx, stokes_idx, cutout_size, output_filename)
